# Gaussian Elimination

이 섹션에서는 가장 직접적인 방법으로 선형 시스템을 푸는 데 도움이 되는 몇 가지 파이썬 함수를 정의하겠습니다.  이 알고리즘은 가우스 제거라고 알려져 있으며, 이 시점부터는 **제거**라고 간단히 부르겠습니다.  제거의 개념은 주어진 시스템을 해는 같지만 풀기가 훨씬 쉬운 다른 시스템과 교환하는 것입니다.  이를 위해 시스템의 해를 보존하면서 점차적으로 해에 더 쉽게 접근할 수 있도록 하는 **행 연산**이라는 일련의 단계를 수행합니다.  이러한 연산에는 세 가지가 있습니다. 
1. 두 방정식의 위치 교환하기
2. 방정식에 0이 아닌 숫자를 곱하기  
3. 방정식을 그 자체의 합과 다른 방정식의 배수로 바꾸기.

## Example 1:  Row operations and elimination

예를 들어 보겠습니다.

$$
\begin{eqnarray*}
x_1 - x_2 + x_3 & = & 3\\
2x_1 + x_2 + 8x_3 & = & 18\\
4x_1 + 2x_2 -3x_3 & = & -2 
\end{eqnarray*}
$$

첫 번째 방정식과 마지막 방정식을 바꿀 수 있습니다.

$$
\begin{eqnarray*}
4x_1 + 2x_2 -3x_3 & = & -2 \\
2x_1 + x_2 + 8x_3 & = & 18\\
x_1 - x_2 + x_3 & = & 3
\end{eqnarray*}
$$

또는 첫 번째 방정식에 $5$를 곱할 수 있습니다.

$$
\begin{eqnarray*}
5x_1 - 5x_2 + 5x_3 & = & 15\\
2x_1 + x_2 + 8x_3 & = & 18\\
4x_1 + 2x_2 -3x_3 & = & -2 
\end{eqnarray*}
$$

또는 마지막 방정식에 첫 번째 방정식의 $2$배를 더할 수도 있습니다.


$$
\begin{eqnarray*}
x_1 - x_2 + x_3 & = & 3\\
2x_1 + x_2 + 3x_3 & = & 8\\
6x_1 \quad\quad -x_3 & = & 4 
\end{eqnarray*}
$$

마지막 연산이 가장 중요한 이유는 방정식 중 하나에서 변수를 '제거'할 수 있기 때문입니다.  세 번째 방정식에는 더 이상 $x_2$ 항이 포함되어 있지 않다는 점에 유의하세요.  이것이 제거 알고리즘의 핵심입니다.

계산을 위해 변수 이름과 "=" 기호를 생략하고 실제 숫자를 모두 배열로 정렬할 수 있습니다.

$$
\begin{equation}
\left[ \begin{array}{rrrr} 1 & -1 & 1 & 3 \\ 2 & 1 & 8 & 18 \\ 4 & 2 & -3 & -2 \end{array}\right]
\end{equation}
$$

이제 이 값으로 NumPy 배열을 만들어 보겠습니다.  나중에 참조할 수 있도록 배열에 $\texttt{A}$라는 이름을 지정하겠습니다.

In [1]:
import numpy as np
A=np.array([[1,-1,1,3],[2,1,8,18],[4,2,-3,-2]])

배열에 대한 연산을 바로 시작할 수도 있지만, 그 대신 먼저 각 연산을 개별적으로 수행하는 몇 가지 코드를 작성하겠습니다.  각 연산을 파이썬 함수 안에 집어넣어 나중에 계산에 다시 사용할 수 있도록 하겠습니다.

In [2]:
def row_swap(A,k,l):
    m = A.shape[0]  # m is number of rows in A
    n = A.shape[1]  # n is number of columns in A
    
    B = np.copy(A).astype('float64')
        
    for j in range(n):
        temp = B[k][j]
        B[k][j] = B[l][j]
        B[l][j] = temp
        
    return B

def row_scale(A,k,scale):
    m = A.shape[0]  # m is number of rows in A
    n = A.shape[1]  # n is number of columns in A
    
    B = np.copy(A).astype('float64')

    for j in range(n):
        B[k][j] *= scale
        
    return B

def row_add(A,k,l,scale):
    m = A.shape[0]  # m is number of rows in A
    n = A.shape[1]  # n is number of columns in A
    
    B = np.copy(A).astype('float64')
        
    for j in range(n):
        B[l][j] += B[k][j]*scale
        
    return B 


이제 $\texttt{row_swap}$, $\texttt{row_scale}$, $\texttt{row_add}$라는 세 가지 새로운 함수가 생겼습니다.  이 함수를 사용해 어떤 결과가 나오는지 살펴봅시다.

In [3]:
B1 = row_swap(A,0,2)
B2 = row_scale(A,2,0.5)
B3 = row_add(A,0,1,2)

In [4]:
print(A)
print('\n')
print(B2)

[[ 1 -1  1  3]
 [ 2  1  8 18]
 [ 4  2 -3 -2]]


[[ 1.  -1.   1.   3. ]
 [ 2.   1.   8.  18. ]
 [ 2.   1.  -1.5 -1. ]]


제거의 목표는 이 배열에 행 연산을 수행하여 다음과 같은 구조의 새 배열을 생성하는 것입니다.

$$
\begin{equation}
\left[ \begin{array}{cccc} 1 & * & * & * \\ 0 & 1 & * & * \\ 0 & 0 & 1 & * \end{array}\right]
\end{equation}
$$

*(여기서 * 기호는 0 또는 1이 될 수도 있고 아닐 수도 있는 다양한 미지의 값을 나타냅니다.)*.

행 연산을 수행하고 각 단계마다 새로운 이름을 붙인 배열로 진행 상황을 저장합니다.  예를 들어 $\texttt{A1}$, $\texttt{A2}$, $\texttt{A3}$ 등으로 이름을 지정할 수 있습니다. 이렇게 하면 진행 상황을 확인하거나 원하는 경우 돌아가서 코드를 변경할 수 있습니다.  

In [5]:
## Add -2 times row 0 to row 1
A1 = row_add(A,0,1,-2)
print(A1,'\n')

## Add -4 times row 0 to row 2
A2 = row_add(A1,0,2,-4)
print(A2,'\n')

## Add -2 times row 1 to row 2
A3 = row_add(A2,1,2,-2)
print(A3,'\n')

## Multiply row 1 by 1/3
A4 = row_scale(A3,1,1.0/3)
print(A4,'\n')

## Multiply row 2 by 1/19
A5 = row_scale(A4,2,1.0/-19.)
print(A5)

[[ 1. -1.  1.  3.]
 [ 0.  3.  6. 12.]
 [ 4.  2. -3. -2.]] 

[[  1.  -1.   1.   3.]
 [  0.   3.   6.  12.]
 [  0.   6.  -7. -14.]] 

[[  1.  -1.   1.   3.]
 [  0.   3.   6.  12.]
 [  0.   0. -19. -38.]] 

[[  1.  -1.   1.   3.]
 [  0.   1.   2.   4.]
 [  0.   0. -19. -38.]] 

[[ 1. -1.  1.  3.]
 [ 0.  1.  2.  4.]
 [-0. -0.  1.  2.]]


이제 이 배열을 모든 기호가 제자리에 있는 시스템 설명으로 다시 변환해 보겠습니다.
$$
\begin{eqnarray*}
x_1 - x_2 + x_3 & = & 3\\
x_2 + 2x_3 & = & 4\\
x_3 & = & 2 
\end{eqnarray*}
$$

제거 단계가 끝나면 우리는 **상부 삼각형** 시스템이라고 알려진 것을 얻게 됩니다.  이 시스템의 해는 마지막 방정식을 거꾸로 풀면 큰 노력 없이 찾을 수 있습니다.  마지막 방정식은 $x_3 = 2$임을 알려줍니다.  이 값을 두 번째 방정식에 대입하면 $x_2 = 0$이 됩니다.  마지막으로, 이 값을 다시 첫 번째 방정식에 대입하면 $x_1 = 1$이 됩니다. 이처럼 위 삼각형 시스템의 해를 구하는 과정을 **역치환**이라고 합니다.

## Example 2: Finding pivots

보시다시피, 마지막 예제의 코드는 배율을 계산하기 위해 나눈 배열의 항목 중 하나라도 0이 나타나면 실패합니다.  이러한 중요한 항목을 **피벗**이라고 하며, 매트릭스에서 해당 항목의 위치를 **피벗 위치**라고 합니다.  정의상 피벗은 0이 아니어야 합니다.  제거 단계에서 피벗 위치에 0이 발생하면 행의 순서를 바꿔서 0이 아닌 항목을 피벗 위치로 옮길 수 있습니다.  무작위 배열에서 작동하는 코드를 작성하기 전에 먼저 특정 배열에 대해 이 방법을 시도해 보겠습니다.

$$
\begin{eqnarray*}
x_1 - x_2 + x_3 & = & 3\\
2x_1 - 2x_2 + 4x_3 & = & 8\\
3x_1 \quad\quad -9x_3 & = & 0 
\end{eqnarray*}
$$


In [6]:
G=np.array([[1,-1,1,3],[2,-2,4,8],[3,0,-9,0]])
print(G)

[[ 1 -1  1  3]
 [ 2 -2  4  8]
 [ 3  0 -9  0]]


In [7]:
## Add -2 times row 0 to row 1
G1 = row_add(G,0,1,-2)
## Add -3 times row 0 to row 2
G2 = row_add(G1,0,2,-3)
print(G2)

[[  1.  -1.   1.   3.]
 [  0.   0.   2.   2.]
 [  0.   3. -12.  -9.]]


이제 중간 피벗 위치에 0이 있습니다.  제거를 계속하기 위해 중간 방정식과 마지막 방정식을 바꿀 수 있습니다.

In [8]:
## Swap rows 1 and 2
G3 = row_swap(G2,1,2)
## Scale the new row 1 by 1/3
G4 = row_scale(G3,1,1./3)
## Scale the new row 2 by 1/2
G5 = row_scale(G4,2,1./2)
print(G5)

[[ 1. -1.  1.  3.]
 [ 0.  1. -4. -3.]
 [ 0.  0.  1.  1.]]


시스템을 익숙한 방정식 집합으로 다시 작성합니다.

$$
\begin{eqnarray*}
x_1 - x_2 + x_3 & = & 3\\
x_2 - 4x_3 & = & -3\\
x_3 & = & 1 
\end{eqnarray*}
$$

역치환을 적용하면 $x_2 = 1$과 $x_1=3$이 됩니다.

행을 바꾸는 것은 정리의 문제로만 필요하다는 점에 주목할 필요가 있습니다.  다음은 제거를 수행하여 동일한 시스템으로 끝낼 수 있는 다른 방법입니다.


In [9]:
## Scale row 1 by 1/2
G3_alternative = row_scale(G2,1,1./2)
## Scale row 2 by 1/3
G4_alternative = row_scale(G3_alternative,2,1./3)
print(G4_alternative)

[[ 1. -1.  1.  3.]
 [ 0.  0.  1.  1.]
 [ 0.  1. -4. -3.]]


생성된 배열은 방정식의 순서는 물론 다르지만 동일한 단순화된 시스템을 나타냅니다.

$$
\begin{eqnarray*}
x_1 - x_2 + x_3 & = & 3\\
x_3 & = & 1 \\
x_2 - 4x_3 & = & -3
\end{eqnarray*}
$$

역치환의 개념은 이러한 형태의 시스템에서도 잘 작동하지만 알고리즘의 구성이 약간 더 복잡해집니다.